In [8]:

import pandas as pd
from datetime import datetime, timedelta
from tvDatafeed import TvDatafeed, Interval
import talib
tv = TvDatafeed()

In [9]:
# หมวดธุรกิจ (Sectors) - 29 หมวด
sectors = [
    # กลุ่มเกษตรและอุตสาหกรรมอาหาร
    "AGRI",      # ธุรกิจการเกษตร
    "FOOD",      # อาหารและเครื่องดื่ม
    
    # กลุ่มสินค้าอุปโภคบริโภค
    "FASHION",   # แฟชั่น
    "HOME",      # ของใช้ในครัวเรือนและสำนักงาน
    "PERSON",    # ของใช้ส่วนตัวและเวชภัณฑ์
    
    # กลุ่มธุรกิจการเงิน
    "BANK",      # ธนาคาร
    "FIN",       # เงินทุนและหลักทรัพย์
    "INSUR",     # ประกันภัยและประกันชีวิต
    
    # กลุ่มสินค้าอุตสาหกรรม
    "AUTO",      # ยานยนต์
    "IMM",       # วัสดุอุตสาหกรรมและเครื่องจักร
    "PAPER",     # กระดาษและวัสดุการพิมพ์
    "PETRO",     # ปิโตรเคมีและเคมีภัณฑ์
    "PKG",       # บรรจุภัณฑ์
    "STEEL",     # เหล็กและผลิตภัณฑ์โลหะ
    
    # กลุ่มอสังหาริมทรัพย์และก่อสร้าง
    "CONMAT",    # วัสดุก่อสร้าง
    "CONS",      # บริการรับเหมาก่อสร้าง
    "PF&REITs",  # กองทุนรวมอสังหาริมทรัพย์และกองทรัสต์
    "PROP",      # พัฒนาอสังหาริมทรัพย์
    
    # กลุ่มทรัพยากร
    "ENERG",     # พลังงานและสาธารณูปโภค
    "MINE",      # เหมืองแร่
    
    # กลุ่มบริการ
    "COMM",      # พาณิชย์
    "HELTH",     # การแพทย์
    "MEDIA",     # สื่อและสิ่งพิมพ์
    "PROF",      # บริการเฉพาะกิจ
    "TOURISM",   # การท่องเที่ยวและสันทนาการ
    "TRANS",     # ขนส่งและโลจิสติกส์
    
    # กลุ่มเทคโนโลยี
    "ETRON",     # ชิ้นส่วนอิเล็กทรอนิกส์
    "ICT"        # เทคโนโลยีสารสนเทศและการสื่อสาร
]

In [10]:
import time
from datetime import datetime

def fetch_with_retry(symbol, exchange='SET', interval=Interval.in_daily, n_bars=5000, max_retries=3, wait_time=20):
    """
    Fetch stock data with retry mechanism
    
    Args:
        symbol: Stock symbol
        exchange: Exchange name
        interval: Time interval
        n_bars: Number of bars to fetch
        max_retries: Maximum number of retry attempts
        wait_time: Wait time in seconds before retry
    
    Returns:
        DataFrame or None if all retries failed
    """
    for attempt in range(max_retries):
        try:
            print(f"Fetching {symbol} (Attempt {attempt + 1}/{max_retries})...")
            stock_data = tv.get_hist(
                symbol=symbol,
                exchange=exchange,
                interval=interval,
                n_bars=n_bars
            )
            
            # Check if data is valid
            if stock_data is not None and not stock_data.empty:
                print(f"✓ Successfully fetched {symbol} - {len(stock_data)} rows")
                return stock_data
            else:
                print(f"✗ No data returned for {symbol}")
                
        except Exception as e:
            print(f"✗ Error fetching {symbol}: {str(e)}")
        
        # Wait before retry (except for the last attempt)
        if attempt < max_retries - 1:
            print(f"Waiting {wait_time} seconds before retry...")
            time.sleep(wait_time)
    
    print(f"⚠ Failed to fetch {symbol} after {max_retries} attempts")
    return None


# Main execution
failed_symbols = []
successful_symbols = []

for symbol in sectors:
    print(f"\n{'='*60}")
    print(f"Processing: {symbol}")
    print(f"{'='*60}")
    
    stock_data = fetch_with_retry(symbol, wait_time=20, max_retries=3)
    
    if stock_data is not None:
        try:
            filename = f'{symbol}.csv'
            stock_data.to_csv(filename)
            print(f"✓ Saved to {filename}")
            successful_symbols.append(symbol)
        except Exception as e:
            print(f"✗ Error saving {symbol}: {str(e)}")
            failed_symbols.append(symbol)
    else:
        failed_symbols.append(symbol)
    
    # Small delay between symbols to avoid rate limiting
    time.sleep(2)

# Summary
print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
print(f"Total symbols: {len(sectors)}")
print(f"Successful: {len(successful_symbols)}")
print(f"Failed: {len(failed_symbols)}")

if failed_symbols:
    print(f"\nFailed symbols: {', '.join(failed_symbols)}")


Processing: AGRI
Fetching AGRI (Attempt 1/3)...
✓ Successfully fetched AGRI - 5000 rows
✓ Saved to AGRI.csv

Processing: FOOD
Fetching FOOD (Attempt 1/3)...
✓ Successfully fetched FOOD - 5000 rows
✓ Saved to FOOD.csv

Processing: FASHION
Fetching FASHION (Attempt 1/3)...
✓ Successfully fetched FASHION - 4694 rows
✓ Saved to FASHION.csv

Processing: HOME
Fetching HOME (Attempt 1/3)...
✓ Successfully fetched HOME - 4694 rows
✓ Saved to HOME.csv

Processing: PERSON
Fetching PERSON (Attempt 1/3)...
✓ Successfully fetched PERSON - 4688 rows
✓ Saved to PERSON.csv

Processing: BANK
Fetching BANK (Attempt 1/3)...
✓ Successfully fetched BANK - 5000 rows
✓ Saved to BANK.csv

Processing: FIN
Fetching FIN (Attempt 1/3)...
✓ Successfully fetched FIN - 5000 rows
✓ Saved to FIN.csv

Processing: INSUR
Fetching INSUR (Attempt 1/3)...
✓ Successfully fetched INSUR - 5000 rows
✓ Saved to INSUR.csv

Processing: AUTO
Fetching AUTO (Attempt 1/3)...
✓ Successfully fetched AUTO - 4695 rows
✓ Saved to AUTO.csv

ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


✗ No data returned for PF&REITs
Waiting 20 seconds before retry...
Fetching PF&REITs (Attempt 2/3)...


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


✗ No data returned for PF&REITs
Waiting 20 seconds before retry...
Fetching PF&REITs (Attempt 3/3)...


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


✗ No data returned for PF&REITs
⚠ Failed to fetch PF&REITs after 3 attempts

Processing: PROP
Fetching PROP (Attempt 1/3)...
✓ Successfully fetched PROP - 5000 rows
✓ Saved to PROP.csv

Processing: ENERG
Fetching ENERG (Attempt 1/3)...
✓ Successfully fetched ENERG - 5000 rows
✓ Saved to ENERG.csv

Processing: MINE
Fetching MINE (Attempt 1/3)...
✓ Successfully fetched MINE - 3972 rows
✓ Saved to MINE.csv

Processing: COMM
Fetching COMM (Attempt 1/3)...
✓ Successfully fetched COMM - 5000 rows
✓ Saved to COMM.csv

Processing: HELTH
Fetching HELTH (Attempt 1/3)...
✓ Successfully fetched HELTH - 5000 rows
✓ Saved to HELTH.csv

Processing: MEDIA
Fetching MEDIA (Attempt 1/3)...
✓ Successfully fetched MEDIA - 4694 rows
✓ Saved to MEDIA.csv

Processing: PROF
Fetching PROF (Attempt 1/3)...
✓ Successfully fetched PROF - 5000 rows
✓ Saved to PROF.csv

Processing: TOURISM
Fetching TOURISM (Attempt 1/3)...
✓ Successfully fetched TOURISM - 4695 rows
✓ Saved to TOURISM.csv

Processing: TRANS
Fetching 